<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Auto-Complete-Beginner/blob/main/NLP_Auto_Complete_Beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import os
import re
import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install -q -U trax

import trax

import trax.fastmath.numpy as np

from trax import layers as tl

     |████████████████████████████████| 637 kB 5.3 MB/s 
     |████████████████████████████████| 4.9 MB 48.0 MB/s 
     |████████████████████████████████| 462 kB 54.8 MB/s 


In [4]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%203/en_US.twitter.txt

--2022-04-20 06:07:04--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%203/en_US.twitter.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3341555 (3.2M) [text/plain]
Saving to: ‘en_US.twitter.txt’

en_US.twitter.txt   100%[===================>]   3.19M  --.-KB/s    in 0.07s   

2022-04-20 06:07:05 (48.1 MB/s) - ‘en_US.twitter.txt’ saved [3341555/3341555]



In [11]:
with open('en_US.twitter.txt', 'r') as f:
  data = f.read()

In [14]:
print(len(data), type(data))

3335477 <class 'str'>


In [12]:
display(data[0:400])

"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)\nWords from a complete stranger! Made my birthday eve"

In [18]:
display(data[-400:])

"tplace jerks off aristocrats until they finally die in the midst of self-love.\n's icon is life.\nwe just had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

In [21]:
def split2Sentences(data):
  
  sentences = data.split('\n')
  sentences = [s.strip() for s in sentences]
  sentences = [s for s in sentences if len(s)>0]
  return sentences

In [33]:
sentences = split2Sentences(data)
sentences[0:10]

['How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.',
 "When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.",
 "they've decided its more fun if I don't.",
 'So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)',
 'Words from a complete stranger! Made my birthday even better :)',
 'First Cubs game ever! Wrigley field is gorgeous. This is perfect. Go Cubs Go!',
 'i no! i get another day off from skool due to the wonderful snow (: and THIS wakes me up...damn thing',
 "I'm coo... Jus at work hella tired r u ever in cali",
 'The new sundrop commercial ...hehe love at first sight',
 'we need to reconnect THIS WEEK']

In [34]:
nltk.word_tokenize(sentences[2])

['they', "'ve", 'decided', 'its', 'more', 'fun', 'if', 'I', 'do', "n't", '.']

In [31]:
tokenize_sentences =lambda sentences: [nltk.word_tokenize(s.lower()) for s in sentences]

In [36]:
tokenize_sentences(sentences)

[['how',
  'are',
  'you',
  '?',
  'btw',
  'thanks',
  'for',
  'the',
  'rt',
  '.',
  'you',
  'gon',
  'na',
  'be',
  'in',
  'dc',
  'anytime',
  'soon',
  '?',
  'love',
  'to',
  'see',
  'you',
  '.',
  'been',
  'way',
  ',',
  'way',
  'too',
  'long',
  '.'],
 ['when',
  'you',
  'meet',
  'someone',
  'special',
  '...',
  'you',
  "'ll",
  'know',
  '.',
  'your',
  'heart',
  'will',
  'beat',
  'more',
  'rapidly',
  'and',
  'you',
  "'ll",
  'smile',
  'for',
  'no',
  'reason',
  '.'],
 ['they', "'ve", 'decided', 'its', 'more', 'fun', 'if', 'i', 'do', "n't", '.'],
 ['so',
  'tired',
  'd',
  ';',
  'played',
  'lazer',
  'tag',
  '&',
  'ran',
  'a',
  'lot',
  'd',
  ';',
  'ughh',
  'going',
  'to',
  'sleep',
  'like',
  'in',
  '5',
  'minutes',
  ';',
  ')'],
 ['words',
  'from',
  'a',
  'complete',
  'stranger',
  '!',
  'made',
  'my',
  'birthday',
  'even',
  'better',
  ':',
  ')'],
 ['first',
  'cubs',
  'game',
  'ever',
  '!',
  'wrigley',
  'field',
 

In [41]:
data_tokenizer = lambda data: tokenize_sentences(split2Sentences(data))

In [46]:
tokenized_data = data_tokenizer(data)

In [47]:
np.random.shuffle(tokenized_data)
train_size = int(len(tokenized_data)*0.8)
train_data, test_data = tokenized_data[:train_size], tokenized_data[train_size:]

In [48]:
print(len(train_data), len(test_data))

38368 9593


In [83]:
def word_counter(tokenized_data):
  word_dic = {}
  for sentence in tokenized_data:
    for token in sentence:
      if token not in word_dic:
        word_dic[token] = 1
      else:
        word_dic[token] += 1
  return word_dic
  # return collections.Counter(tokenized_data)

In [80]:
word_c = word_counter(tokenized_data)

for i, w in enumerate(word_c):
  if i == 20:
    break

  print(w)

i
ca
n't
stand
prideful
people
,
get
the
fuck
over
yourself
.
damn
this
following
limit
!
stop
spackle


In [88]:
def words_with_nplus_frequncy(tokenize_sentences, count_threshold):
  closed_vocab = []
  word_count = word_counter(tokenized_data)
  for k, v in word_count.items():
    if v >= count_threshold:
      closed_vocab.append(k)
  return closed_vocab

In [90]:
temp_closed_vocab = words_with_nplus_frequncy(tokenize_sentences,2)

In [92]:
temp_closed_vocab[: 20]

['i',
 'ca',
 "n't",
 'stand',
 'prideful',
 'people',
 ',',
 'get',
 'the',
 'fuck',
 'over',
 'yourself',
 '.',
 'damn',
 'this',
 'following',
 'limit',
 '!',
 'stop',
 'time']